# Feature Engineering

Using the results of the extract workload, we have our minimally preprocessed data that we can use to craft the following features.

1. **Eye length ratio**: Distance between points 9 and 10 (or distance between points 11 and 12, whichever is larger) over distance between points 8 and 13.
2. **Eye distance ratio**: distance between center of two eyes (points 0 and points 1) over distance between points 8 and 13.
3. **Nose ratio**: Distance between points 15 and 16 over distance between 20 and 21.
4. **Lip size ratio**: Distance between points 2 and 3 over distance between 17 and 18.
5. **Lip length ratio**: Distance between points 2 and 3 over distance between 20 and 21.
6. **Eye-brow length ratio**: Distance between points 4 and 5 (or distance between points 6 and 7 whichever is larger) over distance between 8 and 13.
7. **Aggressive ratio**: Distance between points 10 and 19 over distance between 20 and 21.

## Fetch Extract Workload Results

In [56]:
import pandas as pd

ex_file_path = "../extract/ex_res/ex_res.csv"

ex_df = pd.read_csv(ex_file_path)

ex_df.head()

,index,gender,person_id,neutral,smile,anger,left_light,p_0_x,p_0_y,p_1_x,...,p_17_x,p_17_y,p_18_x,p_18_y,p_19_x,p_19_y,p_20_x,p_20_y,p_21_x,p_21_y
0,0,1,m001,1,0,0,0,328.444,275.496,434.921,...,374.253,395.527,374.253,416.925,373.276,483.314,280.342,404.390,499.835,402.522
1,1,1,m001,0,1,0,0,344.026,270.090,449.912,...,385.070,389.178,386.006,421.015,390.180,491.438,281.390,393.009,511.685,397.247
2,2,1,m001,0,0,1,0,329.170,291.426,436.553,...,376.119,414.293,378.616,440.265,380.614,505.195,279.723,418.289,501.982,416.291
3,3,1,m001,0,0,0,1,345.098,260.392,451.765,...,387.765,367.059,387.765,393.412,387.765,468.706,286.118,383.373,509.490,389.647
4,4,1,m002,1,0,0,0,327.193,263.025,437.671,...,386.155,367.268,386.155,394.529,388.628,472.462,282.728,382.655,499.719,377.464


## Craft Features from Extract Dataframe

In [57]:
import numpy as np
import os

# Function to calculate distance between two points
# Euclidean Distance Formula
def distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

column_names = ['gender', 'person_id', 'neutral', 'smile', 'anger', 'left_light', 
                'EyeLengthRatio', 'EyeDistanceRatio', 'NoseRatio', 'LipSizeRatio', 'LipLengthRatio', 'EyeBrowLengthRatio', 'AggressiveRatio']

# Calculate the features
ex_features_df = pd.DataFrame(columns=column_names)

ex_features_df[['gender', 'person_id', 'neutral', 'smile', 'anger', 'left_light']] = ex_df[['gender', 'person_id', 'neutral', 'smile', 'anger', 'left_light']]

# Eye length ratio: Distance between points 9 and 10 (or distance between points 11 and 12, whichever is larger) over distance between points 8 and 13.
ex_features_df['EyeLengthRatio'] = ex_df.apply(lambda row:
                                               max(distance(row['p_9_x'], row['p_9_y'], row['p_10_x'], row['p_10_y']),
                                                   distance(row['p_11_x'], row['p_11_y'], row['p_12_x'], row['p_12_y']))
                                                   / distance(row['p_8_x'], row['p_8_y'], row['p_13_x'], row['p_13_y']),
                                                   axis=1)

# Eye distance ratio: distance between center of two eyes (points 0 and points 1) over distance between points 8 and 13
ex_features_df['EyeDistanceRatio'] = ex_df.apply(lambda row: 
                                                 distance(row['p_0_x'], row['p_0_y'], row['p_1_x'], row['p_1_y']) / distance(row['p_8_x'], row['p_8_y'], row['p_13_x'], row['p_13_y']),
                                                axis=1)

# Nose ratio: Distance between points 15 and 16 over distance between 20 and 21.
ex_features_df['NoseRatio'] = ex_df.apply(lambda row: 
                                          distance(row['p_15_x'], row['p_15_y'], row['p_16_x'], row['p_16_y']) / distance(row['p_20_x'], row['p_20_y'], row['p_21_x'], row['p_21_y']),
                                          axis=1)

# Lip size ratio: Distance between points 2 and 3 over distance between 17 and 18.
ex_features_df['LipSizeRatio'] = ex_df.apply(lambda row: 
                                          distance(row['p_2_x'], row['p_2_y'], row['p_3_x'], row['p_3_y']) / distance(row['p_17_x'], row['p_17_y'], row['p_18_x'], row['p_18_y']),
                                          axis=1)

# Lip length ratio: Distance between points 2 and 3 over distance between 20 and 21.
ex_features_df['LipLengthRatio'] = ex_df.apply(lambda row: 
                                          distance(row['p_2_x'], row['p_2_y'], row['p_3_x'], row['p_3_y']) / distance(row['p_20_x'], row['p_20_y'], row['p_21_x'], row['p_21_y']),
                                          axis=1)

# Eye-brow length ratio: Distance between points 4 and 5 (or distance between points 6 and 7 whichever is larger) over distance between 8 and 13.
ex_features_df['EyeBrowLengthRatio'] = ex_df.apply(lambda row:
                                            max(distance(row['p_4_x'], row['p_4_y'], row['p_5_x'], row['p_5_y']),
                                                distance(row['p_6_x'], row['p_6_y'], row['p_7_x'], row['p_7_y']))
                                            / distance(row['p_8_x'], row['p_8_y'], row['p_13_x'], row['p_13_y']),
                                          axis=1)

# Aggressive ratio: Distance between points 10 and 19 over distance between 20 and 21.
ex_features_df['AggressiveRatio'] = ex_df.apply(lambda row: 
                                          distance(row['p_10_x'], row['p_10_y'], row['p_19_x'], row['p_19_y']) / distance(row['p_20_x'], row['p_20_y'], row['p_21_x'], row['p_21_y']),
                                          axis=1)

ex_features_df.to_csv(os.path.join(os.getcwd(), "original_features.csv"), index=True, index_label='index')

In [58]:
test_df = pd.read_csv("original_features.csv")
test_df.head()

,index,gender,person_id,neutral,smile,anger,left_light,EyeLengthRatio,EyeDistanceRatio,NoseRatio,LipSizeRatio,LipLengthRatio,EyeBrowLengthRatio,AggressiveRatio
0,0,1,m001,1,0,0,0,0.193307,0.423768,0.166008,4.474276,0.436174,0.426725,0.944518
1,1,1,m001,0,1,0,0,0.211407,0.428316,0.147227,3.392057,0.469056,0.398077,0.954870
2,2,1,m001,0,0,1,0,0.224320,0.434374,0.155068,3.887081,0.456299,0.414964,0.960188
3,3,1,m001,0,0,0,1,0.183316,0.445299,0.162857,3.382285,0.398878,0.422004,0.925923
4,4,1,m002,1,0,0,0,0.208663,0.480300,0.135395,3.287606,0.412910,0.374373,0.981763


## Model Ready Dataframe

We still require some pre-processing to make our feature-engineered dataframe to be ready for model training.

All of our features values will also perform best when scaled down to the same metric such as [0,1] from MinMax or Z-Score.
From visualizing each of the coordinates, none of the coordinates had an extreme skew that I was able to visually gather, and as such,
I ended up using Z-Score Scaling via the StandardScaler.

- We will encode person_id by numerical labels to represent each person.
- Then, the following columns will be scaled down via the StandardScaler from sklearn, which uses z-score scaling.
    

In [59]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

label_encoder = LabelEncoder()
scaler = StandardScaler()

model_ready_df = ex_features_df.copy()
model_ready_df['person_id'] = label_encoder.fit_transform(model_ready_df['person_id'])

# Columns to be scaled
columns_to_scale = ['EyeLengthRatio', 'EyeDistanceRatio', 'NoseRatio', 'LipSizeRatio', 'LipLengthRatio', 'EyeBrowLengthRatio', 'AggressiveRatio']
model_ready_df[columns_to_scale] = scaler.fit_transform(model_ready_df[columns_to_scale])

model_ready_df.to_csv(os.path.join(os.getcwd(), "model_ready.csv"), index=True, index_label='index')
model_ready_df

,gender,person_id,neutral,smile,anger,left_light,EyeLengthRatio,EyeDistanceRatio,NoseRatio,LipSizeRatio,LipLengthRatio,EyeBrowLengthRatio,AggressiveRatio
0,1,0,1,0,0,0,-0.953240,-1.874377,0.465741,0.941153,-0.570940,2.860458,-0.249554
1,1,0,0,1,0,0,-0.091681,-1.670062,-0.539163,-0.156945,0.134949,1.786591,-0.078427
2,1,0,0,0,1,0,0.522945,-1.397896,-0.119616,0.345342,-0.138899,2.419592,0.009484
3,1,0,0,0,0,1,-1.428806,-0.907017,0.297125,-0.166860,-1.371568,2.683468,-0.556948
4,1,1,1,0,0,0,-0.222307,0.665548,-1.172176,-0.262928,-1.070341,0.898074,0.366138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,0,132,0,0,0,1,-1.434533,2.006732,0.902047,-0.612087,-0.142235,1.326757,0.478381
505,0,133,0,0,1,0,1.031459,0.243960,1.121054,0.860891,-0.322065,0.817728,-0.690946
506,0,133,0,0,0,1,-1.169589,2.003535,-0.003857,-0.495557,-1.542027,2.102731,-1.438994
507,0,134,0,0,1,0,0.562756,-0.856631,1.190556,0.172493,0.617674,-0.912238,1.214776
